# Info

Currently I could't run the training because one epoch takes around 45 min. This is crazy...

I also didn't implement stemming and so on. But the rough outline of the ANN is developed. 

In [107]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

df = pd.read_csv('dataset.csv')

df_target = df['humor']
df_data = df.copy()
df_data.drop(columns='humor')

df_target.head()

0    False
1    False
2     True
3    False
4    False
Name: humor, dtype: bool

In [108]:
from sklearn import preprocessing

#encode target to numeric
label_encoder = preprocessing.LabelEncoder()
df_target = label_encoder.fit_transform(df_target)
df_target

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [122]:
words=20000
max_length=1000

In [137]:
from sklearn.feature_extraction.text import CountVectorizer

#Apply CountVectorizer on text column

#vectorizer = CountVectorizer(binary=True)

#If this vectorizer is used, a word which occurs multiple times is counted and not just displayed as 1 = word occurs
cVectorizer = CountVectorizer(binary=False, max_features=1000) #  restrict feature size --> may need to play around

#At the moment I am not able to apply the vectorizer on the whole dataset. To many records & less available RAM
matrix = cVectorizer.fit_transform(df_data['text']).toarray()

In [138]:
from sklearn.model_selection import train_test_split

df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    matrix, df_target, test_size=0.2, random_state=42)

In [139]:
word_size=words
word_size

embed_size=128

In [140]:
model=tf.keras.Sequential()

# Embedding Layer
model.add(tf.keras.layers.Embedding(word_size, embed_size, input_shape=(df_data_train.shape[1],)))

# LSTM Layer
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

# Output Layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [141]:
type(df_data_test)

numpy.ndarray

In [142]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 1000, 128)         2560000   
                                                                 
 lstm_14 (LSTM)              (None, 128)               131584    
                                                                 
 dense_30 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [143]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [146]:
"""#### Training the model"""

model.fit(df_data_train, df_target_train, epochs=5, batch_size=512)

Epoch 1/5
  5/313 [..............................] - ETA: 45:24 - loss: 0.6848 - accuracy: 0.5824

KeyboardInterrupt: 

In [ ]:


test_loss, test_acurracy = model.evaluate(x_test, y_test)

print("Test accuracy: {}".format(test_acurracy))